In [1]:
library(lme4)
library(sjPlot)
library(Hmisc)

library(gridExtra)
library(ggplot2)

# custom style for tab models
css = list(
    css.thead = 'border-top: double; text-align:center; font-style:normal; font-weight:bold; padding:0.1cm;',
    css.tdata = 'padding:0.05cm; padding-left:0.2cm; padding-right:0.2cm; text-align:left; vertical-align:top;',
    css.depvarhead = 'text-align:center; border-bottom:1px solid; font-style:italic; font-weight:bold',
    css.summary = 'padding-top:0.05cm; padding-bottom:0.05cm;'
  )

Loading required package: Matrix

Loading required package: lattice

Loading required package: survival

Loading required package: Formula

Loading required package: ggplot2


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:base’:

    format.pval, units




# Read Data

In [2]:
wtdata <- read.csv("data/dataset_A_linked.csv")

# filter the relevant analysis period
wtdata = wtdata[wtdata$is_analysis_period,]

In [3]:
colnames(wtdata)

[1] "search_page"             "election_period_cat"    
 [3] "total_results_per_page"  "rank"                   
 [5] "is_selected_result"      "coredomain"             
 [7] "news_representativeness" "is_analysis_period"     
 [9] "visits_TI_B1"            "allnews_visits_TI_B1"   
[11] "engagement_TI_B1"        "visits_TI_C1"           
[13] "allnews_visits_TI_C1"    "engagement_TI_C1"       
[15] "visits_TI_D1"            "allnews_visits_TI_D1"   
[17] "engagement_TI_D1"        "visits_TD_B"            
[19] "allnews_visits_TD_B"     "engagement_TD_B"        
[21] "visits_TD_C"             "allnews_visits_TD_C"    
[23] "engagement_TD_C"         "visits_TD_D"            
[25] "allnews_visits_TD_D"     "engagement_TD_D"        
[27] "participant"             "age"                    
[29] "education"               "political_alignment"    
[31] "is_west_germany"         "is_female"              
[33] "is_diverse"              "age_val"                
[35] "education_val"           "political_alignment_val"
[37] "is_west_germany_val"     "is_female_val"

# Scale and center variables

In [4]:
# scale and center the election periods
wtdata$is_before_elections <- scale(wtdata$election_period_cat == 'A', scale=T)
wtdata$is_during_elections <- scale(wtdata$election_period_cat == 'B', scale=T)
wtdata$is_after_elections <- scale(wtdata$election_period_cat == 'C', scale=T)

# scaling the rest of the variables
wtdata$rank <- scale(wtdata$rank, scale=T)
# invert this value so that the odds ratio, so it the scaled scores
# match the intuition, i.e. the 1st result should have a higher (positive)
# value. The odds ratio are then easier to interpret.
wtdata$inv_rank <- -1 * wtdata$rank

# scale and center the representativeness
wtdata$news_representativeness <- scale(wtdata$news_representativeness, scale=T)

# scale and center engagement variables
wtdata$engagement_TI_B1 <- scale(wtdata$engagement_TI_B1, scale=T)
wtdata$engagement_TI_C1 <- scale(wtdata$engagement_TI_C1, scale=T)
wtdata$engagement_TI_D1 <- scale(wtdata$engagement_TI_D1, scale=T)

# scale and center engagement variables
wtdata$visits_TI_B1 <- scale(wtdata$visits_TI_B1, scale=T)
wtdata$visits_TI_C1 <- scale(wtdata$visits_TI_C1, scale=T)
wtdata$visits_TI_D1 <- scale(wtdata$visits_TI_D1, scale=T)

# scale and center engagement variables
wtdata$allnews_visits_TI_B1 <- scale(wtdata$allnews_visits_TI_B1, scale=T)
wtdata$allnews_visits_TI_C1 <- scale(wtdata$allnews_visits_TI_C1, scale=T)
wtdata$allnews_visits_TI_D1 <- scale(wtdata$allnews_visits_TI_D1, scale=T)

# convert to factors the within subjects
wtdata$search_page <- as.factor(wtdata$search_page)
wtdata$participant <- as.factor(wtdata$participant)
wtdata$coredomain <- as.factor(wtdata$coredomain)

# Dataset B1: previous engagement

In [5]:
# time independent
ti_model_B1 <- glmer(is_selected_result ~ 

                            # main factors
                              visits_TI_B1
                            * inv_rank
                            * news_representativeness 
                     
                            # control by total news visits
                            + allnews_visits_TI_B1

                            # control for demographics
                            + is_female
                            + is_west_germany
                            + age
                            + education
                            + political_alignment

                            # control for election day
                            + is_during_elections
                            + is_after_elections

                            # within subject controls
                            + (1|search_page)
                            + (1|coredomain)
                            + (1|participant)

                            , data=wtdata, family=binomial)

# save the model and display
htmlmodel <- tab_model(ti_model_B1, wrap.labels = F, CSS = css)
writeLines(text = htmlmodel$page.complete, con = "statistical_tests/absolute_visits/ti_model_B1.html")
IRdisplay::display_html( htmlmodel$page.content)

boundary (singular) fit: see help('isSingular')



is_selected_result 
 
 
 Predictors 
 Odds Ratios 
 CI 
 p 
 
 
 (Intercept) 
 0.22 
 0.20 – 0.26 
 <0.001 
 
 
 visits TI B1 
 0.87 
 0.72 – 1.05 
 0.151 
 
 
 inv rank 
 2.96 
 2.61 – 3.35 
 <0.001 
 
 
 news representativeness 
 0.76 
 0.65 – 0.90 
 0.001 
 
 
 allnews visits TI B1 
 1.03 
 0.93 – 1.14 
 0.549 
 
 
 is female 
 0.98 
 0.88 – 1.08 
 0.663 
 
 
 is west germany 
 0.94 
 0.85 – 1.04 
 0.210 
 
 
 age 
 0.95 
 0.85 – 1.07 
 0.404 
 
 
 education 
 0.96 
 0.86 – 1.07 
 0.440 
 
 
 political alignment 
 1.05 
 0.95 – 1.16 
 0.344 
 
 
 is during elections 
 0.95 
 0.85 – 1.06 
 0.362 
 
 
 is after elections 
 1.04 
 0.93 – 1.16 
 0.529 
 
 
 visits TI B1 * inv rank 
 1.05 
 0.86 – 1.27 
 0.632 
 
 
 visits TI B1 * news representativeness 
 0.77 
 0.52 – 1.13 
 0.180 
 
 
 inv rank * news representativeness 
 1.08 
 0.91 – 1.27 
 0.379 
 
 
 (visits TI B1 * inv rank) * news representativeness 
 1.19 
 0.85 – 1.67 
 0.320 
 
 
 Random Effects 
 

 
 σ 2 
 3.29 
 

 
 τ 00 search_page 
 0.00 

 
 τ 00 participant 
 0.00 

 
 τ 00 coredomain 
 0.00 

 
 N search_page 
 605 

 
 N coredomain 
 152 

 
 N participant 
 192 
 
 Observations 
 2730 
 
 
 Marginal R 2 / Conditional R 2 
 0.275 / NA

# Dataset C1: Google associated previous engagement

In [6]:
# time independent
ti_model_C1 <- glmer(is_selected_result ~ 

                            # main factors
                              visits_TI_C1
                            * inv_rank
                            * news_representativeness 
                     
                            # control by total news visits
                            + allnews_visits_TI_C1

                            # control for demographics
                            + is_female
                            + is_west_germany
                            + age
                            + education
                            + political_alignment

                            # control for election day
                            + is_during_elections
                            + is_after_elections

                            # within subject controls
                            + (1|search_page)
                            + (1|coredomain)
                            + (1|participant)

                            , data=wtdata, family=binomial)

# save the model and display
htmlmodel <- tab_model(ti_model_C1, wrap.labels = F, CSS = css)
writeLines(text = htmlmodel$page.complete, con = "statistical_tests/absolute_visits/ti_model_C1.html")
IRdisplay::display_html( htmlmodel$page.content)


boundary (singular) fit: see help('isSingular')



is_selected_result 
 
 
 Predictors 
 Odds Ratios 
 CI 
 p 
 
 
 (Intercept) 
 0.22 
 0.20 – 0.26 
 <0.001 
 
 
 visits TI C1 
 0.91 
 0.78 – 1.06 
 0.241 
 
 
 inv rank 
 2.95 
 2.61 – 3.34 
 <0.001 
 
 
 news representativeness 
 0.77 
 0.66 – 0.90 
 0.001 
 
 
 allnews visits TI C1 
 0.99 
 0.89 – 1.10 
 0.828 
 
 
 is female 
 0.98 
 0.88 – 1.08 
 0.640 
 
 
 is west germany 
 0.94 
 0.85 – 1.04 
 0.201 
 
 
 age 
 0.95 
 0.86 – 1.07 
 0.410 
 
 
 education 
 0.96 
 0.86 – 1.07 
 0.430 
 
 
 political alignment 
 1.04 
 0.94 – 1.15 
 0.430 
 
 
 is during elections 
 0.94 
 0.84 – 1.06 
 0.323 
 
 
 is after elections 
 1.04 
 0.93 – 1.16 
 0.521 
 
 
 visits TI C1 * inv rank 
 0.98 
 0.82 – 1.19 
 0.874 
 
 
 visits TI C1 * news representativeness 
 0.96 
 0.78 – 1.18 
 0.695 
 
 
 inv rank * news representativeness 
 1.07 
 0.91 – 1.26 
 0.410 
 
 
 (visits TI C1 * inv rank) * news representativeness 
 1.16 
 0.86 – 1.56 
 0.339 
 
 
 Random Effects 
 

 
 σ 2 
 3.29 
 

 
 τ 00 search_page 
 0.00 

 
 τ 00 participant 
 0.00 

 
 τ 00 coredomain 
 0.00 

 
 N search_page 
 605 

 
 N coredomain 
 152 

 
 N participant 
 192 
 
 Observations 
 2730 
 
 
 Marginal R 2 / Conditional R 2 
 0.266 / NA

# Dataset D1: non Google associated previous engagement

In [7]:
# time independent
ti_model_D1 <- glmer(is_selected_result ~ 

                            # main factors
                              visits_TI_D1
                            * inv_rank
                            * news_representativeness 
                     
                            # control by total news visits
                            + allnews_visits_TI_D1

                            # control for demographics
                            + is_female
                            + is_west_germany
                            + age
                            + education
                            + political_alignment

                            # control for election day
                            + is_during_elections
                            + is_after_elections

                            # within subject controls
                            + (1|search_page)
                            + (1|coredomain)
                            + (1|participant)

                            , data=wtdata, family=binomial)



# save the model and display
htmlmodel <- tab_model(ti_model_D1, wrap.labels = F, CSS = css)
writeLines(text = htmlmodel$page.complete, con = "statistical_tests/absolute_visits/ti_model_D1.html")
IRdisplay::display_html( htmlmodel$page.content)

boundary (singular) fit: see help('isSingular')



is_selected_result 
 
 
 Predictors 
 Odds Ratios 
 CI 
 p 
 
 
 (Intercept) 
 0.22 
 0.20 – 0.26 
 <0.001 
 
 
 visits TI D1 
 0.85 
 0.68 – 1.07 
 0.168 
 
 
 inv rank 
 2.96 
 2.62 – 3.36 
 <0.001 
 
 
 news representativeness 
 0.76 
 0.64 – 0.89 
 0.001 
 
 
 allnews visits TI D1 
 1.04 
 0.94 – 1.15 
 0.453 
 
 
 is female 
 0.98 
 0.89 – 1.08 
 0.684 
 
 
 is west germany 
 0.94 
 0.85 – 1.04 
 0.222 
 
 
 age 
 0.95 
 0.85 – 1.06 
 0.388 
 
 
 education 
 0.96 
 0.86 – 1.07 
 0.425 
 
 
 political alignment 
 1.05 
 0.95 – 1.16 
 0.326 
 
 
 is during elections 
 0.95 
 0.85 – 1.06 
 0.364 
 
 
 is after elections 
 1.04 
 0.93 – 1.16 
 0.530 
 
 
 visits TI D1 * inv rank 
 1.08 
 0.87 – 1.35 
 0.490 
 
 
 visits TI D1 * news representativeness 
 0.71 
 0.43 – 1.16 
 0.168 
 
 
 inv rank * news representativeness 
 1.08 
 0.92 – 1.28 
 0.346 
 
 
 (visits TI D1 * inv rank) * news representativeness 
 1.23 
 0.82 – 1.84 
 0.315 
 
 
 Random Effects 
 

 
 σ 2 
 3.29 
 

 
 τ 00 search_page 
 0.00 

 
 τ 00 participant 
 0.00 

 
 τ 00 coredomain 
 0.00 

 
 N search_page 
 605 

 
 N coredomain 
 152 

 
 N participant 
 192 
 
 Observations 
 2730 
 
 
 Marginal R 2 / Conditional R 2 
 0.282 / NA